In [1]:
!pip install binance_historical_data mplfinance stable-baselines3 backtesting tensorboard binance ta line_profiler[ipython] scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
#!git clone https://github.com/Klok-e/python-price-predict-experiment.git

import sys

sys.path.insert(0, 'python-price-predict-experiment')

In [3]:
import numpy as np
import pandas as pd
from util import download_and_process_data_if_available

df_tickers = download_and_process_data_if_available("cache/df_tickers.pkl")

Loading data from cache


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# import time
# import os
# try:
#   os.makedirs("./drive/MyDrive/colab/model-weights/")
# except:
#   pass
# my_model.save(f"./drive/MyDrive/colab/model-weights/model-weights-{time.time()}.pt")

In [5]:
#%load_ext tensorboard
#%tensorboard --logdir tensorboard

In [6]:
from train_model import train_model
from env import CustomEnv

from stable_baselines3.common.env_checker import check_env

env = CustomEnv(df_tickers)
check_env(env)
del env

hidden_size_list = [512, 256, 128, 64]
lstm_layers_list = [1, 2]
arch_list = [[256, 256, 256]]
window_size_list = [8, 32, 64, 128]
for hidden_size in hidden_size_list:
    for lstm_layers in lstm_layers_list:
        for arch in arch_list:
            for window_size in window_size_list:
                print(f"hidden_size {hidden_size}, lstm_layers {lstm_layers}, window_size {window_size}")
                train_model(df_tickers, hidden_size, lstm_layers, arch, 100_000_000, window_size)

obs length = {'prices_sequence': (64, 12), 'other': (2,)}
ticker 0; index 1428894
ticker 0; index 435768
ticker 1; index 148151
hidden_size 512, lstm_layers 1, window_size 8


/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:251: UserWarning: Your observation prices_sequence has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


Using cuda device
Logging to ./tensorboard/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.02e+03 |
|    ep_rew_mean     | -72.9    |
| time/              |          |
|    fps             | 3243     |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 10240    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.02e+03    |
|    ep_rew_mean          | -493        |
| time/                   |             |
|    fps                  | 1237        |
|    iterations           | 2           |
|    time_elapsed         | 16          |
|    total_timesteps      | 20480       |
| train/                  |             |
|    approx_kl            | 0.013055295 |
|    clip_fraction        | 0.154       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance

Process ForkServerProcess-2:
Process ForkServerProcess-4:
Process ForkServerProcess-6:
Process ForkServerProcess-1:
Process ForkServerProcess-3:
Process ForkServerProcess-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/storage/dima/Doc/Documents/Особисте/Проєкти/Projects/price_prediction_experiment/venv/lib/python3.11/site-packages/stable_baselines3/common/vec_env/subproc_vec_env.py", line 33, in _worker
    cmd, data = remote.recv()
                ^^^^^^

KeyboardInterrupt: 

In [ ]:
evaluations = np.load("rl-model-best-model/results/evaluations.npz")
evaluations_df = pd.DataFrame()
evaluations_df["timesteps"] = evaluations["timesteps"]
evaluations_df["results"] = evaluations["results"].mean(axis=1)
evaluations_df